# Compute RNA QC metrics

In [ ]:
# Load libraries
import muon as mu
import scanpy as sc
from snakemake.script import snakemake

In [ ]:
# Read input and output paths from Snakemake
input_file = snakemake.input[0]
output_file = snakemake.output[0]

In [ ]:
# Load the data
mdata = mu.read(input_file)
mdata

In [ ]:
# Focus on the RNA modality
adata = mdata.mod["rna"]
adata

In [ ]:
# Mark mitochondrial, ribosomal, and hemoglobin genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
adata

In [ ]:
# Calculate quality control metrics
sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=["mt", "ribo", "hb"],
    percent_top=None,
    inplace=True,
    log1p=False,
)
adata

In [ ]:
# Inspect violin plots of QC metrics
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
# Consider QC metrics jointly by inspecting a scatter plot colored by `pct_counts_mt`
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
# Doublet detection
sc.pp.scrublet(adata)
adata

In [ ]:
# Save updated mdata
adata.write(output_file)